<a href="https://colab.research.google.com/github/gmb408/OSSP-KBO-prediction/blob/version-0.1/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 데이터 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

In [5]:
result = pd.read_csv("/content/statiz_pre.csv", index_col=0)


In [6]:
# 단타 feature 생성
result['1타'] = result['안타'] - result['2타'] - result['3타'] - result['홈런']

In [7]:
# 사용 features 명시
infos = ['ID','이름','생일','팀','시즌','포지션','나이']
stats = ['G','타수','득점','안타','1타','2타','3타','홈런','루타','타점','사사구','삼진','타율','출루','장타','OPS','wOBA','wRC+','WAR+']
result['사사구'] = result['볼넷'] + result['사구']
result = result[infos+stats]

In [8]:
# 해당 시즌 이전까지의 누적 기록
def get_cumulative(player_id, season, stat=stats ,df=result):
    try:
        result_cum = df[(df['ID']==player_id) & (df['시즌'] < season)][stat]
        result_cum = result_cum.mean()
        
    except:
        result_cum = [np.nan for i in stat]
        result_cum = pd.Series(result_cum)
    
    return result_cum

In [9]:
# 모든 stats에 대한 누적 stat
stats_cum = [i + '_누적' for i in stats]

In [10]:
# 누적 stat 데이터프레임에 반영
result[stats_cum] = result.apply(lambda x: get_cumulative(x['ID'], x['시즌']), axis=1)

In [11]:
# lag stat (직전 몇년의 기록)
def get_lag(player_id, season, lag, stat=stats, df=result):
    try:
        result_lag = df[(df['ID']==player_id)&(df['시즌'] == season-lag)][stat].values[0]
        result_lag = pd.Series(result_lag)
        
    except:
        result_lag = [np.nan for i in stat]
        result_lag = pd.Series(result_lag)
        
    return result_lag


In [12]:

# 직전 1,2,3년의 기록
stats_lag1 = [i + '_LAG1' for i in stats]
stats_lag2 = [i + '_LAG2' for i in stats]
stats_lag3 = [i + '_LAG3' for i in stats]

In [13]:
# lag1
result[stats_lag1] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=1), axis=1)

In [14]:
# lag2
result[stats_lag2] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=2), axis=1)

In [15]:
# lag3
result[stats_lag3] = result.apply(lambda x: get_lag(x['ID'],x['시즌'], lag=3), axis=1)

In [16]:
# 내년 기록 (OPS, 장타, 출루)
# 데이터 프레임의 정답 셋 만들기 위함

def get_next_year(player_id, season, stat=['장타','출루','OPS'],df=result):
    try:
        result = df[(df['ID']==player_id)&(df['시즌']==season+1)][stat].values[0]
        result = pd.Series(result)
    except:
        result = [np.nan for i in stat]
        result = pd.Series(result)
        
    return result

In [17]:
# 장타, 출루, OPS의 내년 기록
stats_next = [i + '_NEXT' for i in ['장타', '출루', 'OPS']]
result[stats_next] = result.apply(lambda x: get_next_year(x['ID'], x['시즌']), axis=1)

In [18]:
# 변환 데이터 저장
result.to_csv("/content/statiz_Final.csv")
result

,ID,이름,생일,팀,시즌,포지션,나이,G,타수,득점,...,타율_LAG3,출루_LAG3,장타_LAG3,OPS_LAG3,wOBA_LAG3,wRC+_LAG3,WAR+_LAG3,장타_NEXT,출루_NEXT,OPS_NEXT
0,0,이종범,1970-08-15,해,1994,SS,25,124.0,499.0,113.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.586,0.397,0.983
1,1,테임즈,1986-11-10,N,2015,1B,30,142.0,472.0,130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.679,0.427,1.106
2,2,심정수,1975-05-05,현,2003,RF,29,133.0,460.0,110.0,...,0.304,0.400,0.551,0.951,0.413,149.9,5.31,NaN,NaN,NaN
3,0,이종범,1970-08-15,해,1997,SS,28,125.0,484.0,112.0,...,0.393,0.452,0.581,1.033,0.462,198.3,11.77,NaN,NaN,NaN
4,0,이종범,1970-08-15,해,1996,SS,27,113.0,449.0,94.0,...,0.280,0.331,0.432,0.762,0.350,127.4,6.14,0.581,0.428,1.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3863,2187,임종찬,2001-09-28,한,2021,RF,21,42.0,132.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3864,1838,박찬호,1995-06-05,K,2020,SS,26,141.0,479.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.313,0.331,0.644
3865,1993,정보근,1999-08-31,롯,2020,C,22,85.0,133.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3866,1801,나종덕,1998-03-16,롯,2019,C,22,104.0,185.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
